# CrystaLLM Tutorial Practical

In this practical we will download a pre-trained version of `CrystaLLM` we will use this to generate some potential crystal structures based on text prompts of the formula, and the formula with a spacegroup.

# Note - rememeber to enable GPU for this practical

If you do not enable GPU, then later on you will need to specify to `CrystaLLM` to run on CPU.

## Set up the code

We clone the git repo.

In [1]:
import os

os.chdir('/content')
if not os.path.exists('CrystaLLM'):
  !git clone https://github.com/lantunes/CrystaLLM.git

os.chdir('CrystaLLM')

Cloning into 'CrystaLLM'...
remote: Enumerating objects: 1328, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 1328 (delta 251), reused 235 (delta 232), pack-reused 1039 (from 1)
Receiving objects: 100% (1328/1328), 51.52 MiB | 15.72 MiB/s, done.
Resolving deltas: 100% (914/914), done.
Updating files: 100% (71/71), done.


## Note we need to run this next cell twice for some reason!

Install the requirements for `CrystaLLM`

In [3]:
pip install -r requirements.txt --quiet

Now put `CrystaLLM` on the PythonPath

In [30]:
pip install -e .

Obtaining file:///content/CrystaLLM
  Preparing metadata (setup.py) ... done
  Attempting uninstall: crystallm
    Found existing installation: crystallm 1.0
    Uninstalling crystallm-1.0:
      Successfully uninstalled crystallm-1.0
  Running setup.py develop for crystallm


## Obtain the pretrained model

The pretrained `CrystaLLM` as published in XXXX is available to download from Zenodo. There is a helpful `bin/download.py` script to help you with this.

We download the small model (~25M parameters). But you can also access the larger model using `!tar xvf crystallm_v1_large.tar.gz`. In addition there are other models fror download, which are trained on different datasets, for the full list see the [config directory](https://github.com/lantunes/CrystaLLM/tree/main/config) of the repo.

In [5]:
!python bin/download.py crystallm_v1_small.tar.gz
!tar xvf crystallm_v1_small.tar.gz

downloading to ./crystallm_v1_small.tar.gz ...
100% 285M/285M [02:03<00:00, 2.31MiB/s]
done!


## Generate a prompt

`CrystaLLM` needs a prompt to start generating a file. This prompt is the opening text of the file. At its simplest we can just give a chemical formula. We put the prompt into a `.txt` file that will be read by the run script later on. We could also add a spacegroup using the `--spacegroup` option:

```
python bin/make_prompt_file.py Na2Cl2 my_sg_prompt.txt --spacegroup P4/nmm
```

In [27]:
!python bin/make_prompt_file.py CaTiO3 my_prompt.txt

writing prompt to my_prompt.txt ...


## Run `CrystaLLM`


To randomly sample from a trained model, and generate CIF files, use the `bin/sample.py` script. The sampling script
expects the path to the folder containing the trained model checkpoint, as well as the prompt, and other configuration
options.

<details>
  <summary>Click for supported configuration options and their default values</summary>

  ```python
  out_dir: str = "out"  # the path to the directory containing the trained model
  start: str = "\n"  # the prompt; can also specify a file, use as: "FILE:prompt.txt"
  num_samples: int = 2  # number of samples to draw
  max_new_tokens: int = 3000  # number of tokens generated in each sample
  temperature: float = 0.8  # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
  top_k: int = 10  # retain only the top_k most likely tokens, clamp others to have 0 probability
  seed: int = 1337
  device: str = "cuda"  # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
  dtype: str = "bfloat16"  # 'float32' or 'bfloat16' or 'float16'
  compile: bool = False  # use PyTorch 2.0 to compile the model to be faster
  target: str = "console"  # where the generated content will be sent; can also be 'file'
  ```

</details>

For example:
```shell
python bin/sample.py \
out_dir=out/my_model \
start=FILE:my_prompt.txt \
num_samples=2 \
top_k=10 \
max_new_tokens=3000 \
device=cuda
```
In the above example, the trained model checkpoint file exists in the `out/my_model` directory. The prompt is
in a file located at `my_prompt.txt`. Alternatively, we could also have placed the configuration options in a .yaml
file, as we did for training, and specified its path using the `--config` command line option.

Instead of specifying a file containing the prompt, we could also have specified the prompt directly:
```shell
python bin/sample.py \
out_dir=out/my_model \
start=$'data_Na2Cl2\n' \
num_samples=2 \
top_k=10 \
max_new_tokens=3000 \
device=cuda
```
Assuming we're in a bash environment, we use the `$'string'` syntax for the `start` argument, since we'd like to
specify the `\n` (new line) character at the end of the prompt.

The generated CIF files are sent to the console by default. Include the `target=file` argument to save the generated CIF
files locally. (Each file will be named `sample_1.cif`, `sample_2.cif`, etc.)

In [28]:
! python bin/sample.py \
out_dir=crystallm_v1_small/ \
start=FILE:my_prompt.txt \
num_samples=2 \
top_k=10 \
max_new_tokens=3000 \
device=cuda \
target=file

Using configuration:
out_dir: crystallm_v1_small/
start: FILE:my_prompt.txt
num_samples: 2
max_new_tokens: 3000
temperature: 0.8
top_k: 10
seed: 1337
device: cuda
dtype: bfloat16
compile: false
target: file

/content/CrystaLLM/bin/sample.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have f

## Visualise the results

Use ASE to see what our outputs looked like

In [29]:
import ase.io
from ase.visualize import view
from ase.build import make_supercell
import numpy as np

structure = ase.io.read('sample_2.cif')
supercell = 3
print('Formula: ', structure.symbols)
print('Unit cell: ', structure.cell)
view(make_supercell(structure, supercell * np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])), viewer='x3d')

Formula:  CaTiO
Unit cell:  Cell([3.822, 3.822, 3.822])


## Exercises

* Try generating other structures
* Try doing sample generation with spacegroup as well as composition
* Load one of the other models - e.g. `crystallm_carbon_24_small` this has been trained only on allotropes of carbon. How good is this at generating a perovskite structure?
* Try out the large model - do the results look different to the small model?
* You can try to do generation using Monte Carlo Tree Search to choose conditioned next tokens, see [the documentation here](https://github.com/lantunes/CrystaLLM?tab=readme-ov-file#monte-carlo-tree-search-decoding) - in principle this should lead to lower energy genearated structures. See how it affects your generations.